# Pre-requisites  
- download dataset.zip,  
- unzip dataset,    
- mount drive,  

In [1]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import pdb
# import numpy as np
# import torchvision.models as models


# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.model = models.resnet50(pretrained=True)
#         self.num_ftrs = self.model.fc.in_features
#         modules = list(self.model.children())[:-1]
#         self.model = nn.Sequential(*modules)
#         self.normalize = Normalize()
#         return

#     def forward(self, x):
#         return self.forward_as_dict(x)['conv6']

#     def forward_as_dict(self, x):
#         # pdb.set_trace()

#         x = self.conv1a(x)

#         x = self.b2(x)
#         x = self.b2_1(x)
#         x = self.b2_2(x)

#         x = self.b3(x)
#         x = self.b3_1(x)
#         x = self.b3_2(x)

#         x = self.b4(x)
#         x = self.b4_1(x)
#         x = self.b4_2(x)
#         x = self.b4_3(x)
#         x = self.b4_4(x)
#         x = self.b4_5(x)

#         x, conv4 = self.b5(x, get_x_bn_relu=True)
#         x = self.b5_1(x)
#         x = self.b5_2(x)

#         x, conv5 = self.b6(x, get_x_bn_relu=True)

#         x = self.b7(x)
#         conv6 = F.relu(self.bn7(x))
#         # pdb.set_trace()

#         return dict({'conv4': conv4, 'conv5': conv5, 'conv6': conv6})


#     def train(self, mode=True):

#         super().train(mode)

#         for layer in self.not_training:

#             if isinstance(layer, torch.nn.Conv2d):
#                 layer.weight.requires_grad = False

#             elif isinstance(layer, torch.nn.Module):
#                 for c in layer.children():
#                     c.weight.requires_grad = False
#                     if c.bias is not None:
#                         c.bias.requires_grad = False

#         for layer in self.modules():

#             if isinstance(layer, torch.nn.BatchNorm2d):
#                 layer.eval()
#                 layer.bias.requires_grad = False
#                 layer.weight.requires_grad = False

#         return



# class Net(network.resnet50d.Net):
#     def __init__(self):
#         super(Net, self).__init__()
        
#         self.dropout7 = torch.nn.Dropout2d(0.3)

#         self.k = k_cluster
#         self.from_round_nb = from_round_nb

#         # class 20
#         if self.from_round_nb == 0:

#             self.fc8 = nn.Conv2d(num_ftrs, 8, 1, bias=False)

#             torch.nn.init.xavier_uniform_(self.fc8.weight)
#             self.from_scratch_layers = [self.fc8]

#         # class 20 + class 200
#         else:
#             self.fc8_8 = nn.Conv2d(4096, 8, 1, bias=False)
#             self.fc8_80 = nn.Conv2d(4096, self.k*8, 1, bias=False)

#             torch.nn.init.xavier_uniform_(self.fc8_8.weight)
#             torch.nn.init.xavier_uniform_(self.fc8_80.weight)

#             self.from_scratch_layers = [self.fc8_8, self.fc8_80]


#     def forward(self, x, from_round_nb):
#         x = super().forward(x)
#         x = self.dropout7(x)
#         x = F.avg_pool2d(x, kernel_size=(x.size(2), x.size(3)), padding=0)

#         feature = x
#         feature = feature.view(feature.size(0), -1)

#         # class 20
#         if from_round_nb == 0:
#             x = self.fc8(x)
#             x = x.view(x.size(0), -1)
#             y = torch.sigmoid(x)
#             return x, feature, y

#         # class 20 + class 200
#         else:
#             x_8 = self.fc8_8(x)
#             x_8 = x_8.view(x_8.size(0), -1)
#             y_8 = torch.sigmoid(x_8)


#             x_80 = self.fc8_80(x)
#             x_80 = x_80.view(x_80.size(0), -1)
#             y_80 = torch.sigmoid(x_80)

#             return x_8, feature, y_8, x_80, y_80



#     def multi_label(self, x):
#         x = torch.sigmoid(x)
#         tmp = x.cpu()
#         tmp = tmp.data.numpy()
#         _, cls = np.where(tmp>0.5)

#         return cls, tmp


#     def forward_cam(self, x):
#         x = super().forward(x)
#         x = F.conv2d(x, self.fc8.weight)
#         x = F.relu(x)

#         return x


#     def forward_two_cam(self, x):
#         x_ = super().forward(x)

#         x_8 = F.conv2d(x_, self.fc8_8.weight)
#         cam_8 = F.relu(x_8)

#         x_80 = F.conv2d(x_, self.fc8_80.weight)
#         cam_80 = F.relu(x_80)

#         return cam_8, cam_80

#     def get_parameter_groups(self):
#         groups = ([], [], [], [])

#         for m in self.modules():

#             if isinstance(m, nn.Conv2d):

#                 if m.weight.requires_grad:
#                     if m in self.from_scratch_layers:
#                         groups[2].append(m.weight)
#                     else:
#                         groups[0].append(m.weight)

#                 if m.bias is not None and m.bias.requires_grad:

#                     if m in self.from_scratch_layers:
#                         groups[3].append(m.bias)
#                     else:
#                         groups[1].append(m.bias)

#         return groups


In [2]:
# Package installations
from IPython.utils import io
with io.capture_output() as captured:
    !pip install gdown
    !pip3 install pillow==8.2.0

    #Install neptune.ai client for monitoring the training process
    !pip install neptune-client
    !pip install neptune-client neptune-tensorboard
    !pip install neptune-contrib
    !pip install imgaug==0.2.6
    !pip install mxnet


In [3]:
# Download VOC2012
# !gdown https://drive.google.com/uc?id=1PDTEuTnWJZNWogxYdqYGOlEZHK8dYET9

# Download custom Kvasir-v2
# ! gdown https://drive.google.com/uc?id=1WG5F7VVQe6mNupR1LLirkG_Apx6Bup5J

In [4]:
# # Download .params weights for training
# %cd /content/drive/MyDrive/MAI/thesis/source/SC-CAM/pretrained_model/
# !gdown https://drive.google.com/uc?id=1YB3DkHiBeUH5wn6shk93jChvXwfOxwBE

In [5]:
# with io.capture_output() as captured:
#   # !unzip -q VOC2012.zip

# !unzip -q Kvasir-v2.zip
#   # !rm -rf VOC2012.zip
  # !rm -rf Kvasir-v2.zip

In [6]:
import neptune
from neptunecontrib.monitoring.keras import NeptuneMonitor
from IPython.utils import io
# connect run to project
NEPTUNE_TOKEN = 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJhMGUxMmQ1NC00ZDU4LTQ4ZGYtOWJjOC0xYTJkYjJmYmJiZDMifQ=='
run = neptune.init(project_qualified_name='ch.kalavritinos/SC-CAM', api_token=NEPTUNE_TOKEN)

In [7]:
with io.capture_output() as captured:
  from google.colab import drive 
  drive.mount('/content/drive')
%cd /content/drive/MyDrive/MAI/thesis/source

/content/drive/MyDrive/MAI/thesis/source


In [8]:
import PIL
PIL.__version__

'8.2.0'

In [9]:
import torch
torch.cuda.is_available()

False

In [ ]:
# !unzip kvasir-dataset-v2.zip

# SC-CAM train
First, try to run the scripts

In [10]:
%cd SC-CAM/

/content/drive/MyDrive/MAI/thesis/source/SC-CAM


In [11]:
from torchvision import transforms

In [12]:
import numpy as np
import torch
import torch.nn as nn
from torch.backends import cudnn
cudnn.enabled = True
import voc12.data
import scipy.misc
import importlib
from torch.utils.data import DataLoader, Dataset
import torchvision
from utils import imutils, pyutils
import argparse
from PIL import Image
import torch.nn.functional as F
import os
import matplotlib.pyplot as plt

In [13]:
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        return super(ImageFolderWithPaths, self).__getitem__(index), self.imgs[index][0]

In [14]:
class MyLazyDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __getitem__(self, index):
        if self.transform:
            x = self.transform(self.dataset[index][0])
        else:
            x = self.dataset[index][0]
        y = self.dataset[index][1]
        
        return x, y

    def __len__(self):
        return len(self.dataset)

    def set_transform(self, transform):
        self.transform = transform

In [15]:
class Sub_Class_Dataset(Dataset):
    def __init__(self, voc12_root, crop_size, round_nb, k_cluster, save_folder, test=False):
        print('############################################## || k_{} / Round {} || ##############################################'.format(k_cluster, round_nb))

        self.voc12_root = voc12_root
        self.crop_size = crop_size

        with open('{}/label/R{}_train_filename_list.txt'.format(save_folder, round_nb), 'r') as f:
            self.filename = f.read().split('\n')
        f.close()
        self.filename = self.filename[:-1]  # 16458

        self.label = np.load('{}/label/R{}_train_label_200.npy'.format(save_folder, round_nb))
        self.label = torch.from_numpy(self.label).float()

        self.label_20 = np.load('{}/label/R{}_train_label_20.npy'.format(save_folder, round_nb)) # 16458

        self.label_20 = torch.from_numpy(self.label_20).float()

        print('=='*60)
        print('Training Data: image: {} | 20 class label: {} | 200 class label: {}'.format(len(self.filename), self.label_20.shape, self.label.shape))
        print('=='*60)


    def __getitem__(self, index):
        label_200 = self.label[index]
        label_20 = self.label_20[index]

        self.dataset_path = os.path.join(self.voc12_root, 'JPEGImages')

        filename = self.filename[index]

        img = Image.open(get_img_path(filename, self.dataset_path)).convert("RGB")
        img = imutils.ResizeLong(img, 256, 512)
        img = imutils.Flip(img)
        img = imutils.ColorJitter(img)
        img = np.array(img)
        img = imutils.NNormalize(img)
        img = imutils.Crop(img, self.crop_size)
        img = img.transpose(2,0,1)
        img = torch.from_numpy(img)

        return img, label_20, label_200, filename

    def __len__(self):
        return len(self.filename)

In [16]:
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __init__(self, root, transform, round_nb, save_folder='./results_kvasir', test=False):
        super(ImageFolderWithPaths, self).__init__(root, transform)
        self.root = root
        self.label_200 = np.load('{}/label/R{}_train_label_200.npy'.format(save_folder, round_nb))
        self.label_200 = torch.from_numpy(self.label_200).float()

        self.label_20 = np.load('{}/label/R{}_train_label_20.npy'.format(save_folder, round_nb)) # 16458
        self.label_20 = torch.from_numpy(self.label_20).float()


    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        label_200 = self.label_200[index]
        label_20 = self.label_20[index]
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path, label_20, label_200))
        return tuple_with_path

def get_data_loader(data_dir, round_nb=1, batch_size=32, train=True):
    transform = {
        'train': transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(256),
            transforms.Resize(224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.RandomRotation(90),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ]),
        'test': transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(256),
            transforms.Resize(224),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ]),
    }

    # ImageFolder with root directory and defined transformation methods for batch as well as data augmentation
    if train:
      data = ImageFolderWithPaths(root=data_dir, transform=transform['train'], round_nb=round_nb)
    else:
      data = ImageFolderWithPaths(root=data_dir, transform=transform['test'], round_nb=round_nb)
    data_loader = torch.utils.data.DataLoader(dataset=data, batch_size=batch_size, shuffle=False, num_workers=2)

    return data.class_to_idx, data_loader

In [17]:
def make_folder(save_folder_path):
    if os.path.exists(save_folder_path) == False:
        os.mkdir(save_folder_path)
    if os.path.exists(os.path.join(save_folder_path, 'feature')) == False:
        os.mkdir(os.path.join(save_folder_path, 'feature'))
    if os.path.exists(os.path.join(save_folder_path, 'label')) == False:
        os.mkdir(os.path.join(save_folder_path, 'label'))
    if os.path.exists(os.path.join(save_folder_path, 'log')) == False:
        os.mkdir(os.path.join(save_folder_path, 'log'))
    if os.path.exists(os.path.join(save_folder_path, 'weight')) == False:
        os.mkdir(os.path.join(save_folder_path, 'weight'))

# PARAMETERS

In [44]:
root_dir = 'Kvasir'
data_root = '/content/drive/MyDrive/MAI/thesis/source/kvasir-dataset-v2'   # dataset directory
# data_root = '/content/drive/MyDrive/MAI/thesis/source/VOCdevkit/VOC2012/'


weights = "./pretrained_model/res38_cls.pth"
network = "network.resnet38_cls"
infer_list = "voc12/val.txt"
from_round_nb = 0
k_cluster = 10
save_folder = './results_kvasir'

make_folder(save_folder)

In [ ]:
epoch = 25
lr = 0.01
batch_size = 8
input_size = 256
disp_interval = 100
num_classes = 8
num_workers = 2
weight_decay = 0.0005
decay_points = '5,10'

crop_size = 224         # for cropping images
threshold = 0.6         # for probabilities
disp_interval = 100     # display interval
resume  = False         # resume training 
global_counter = 0      
current_epoch = 0       # number of current epoch for resuming

# imutils.py


In [18]:
import PIL.Image
import random
import numpy as np

"""Modified classes to include saliency maps as well
        Classes Included:
            RandomResizeLong()
            RandomCrop()
            AvgPool2d()
            RandomHorizontalFlip()
            CenterCrop()
            RescaleNearest()

        Functions included: 
            get_random_crop_box()
            crop_with_box()
            random_crop()
            HWC_to_CHW()
            crf_inference()
            bb_IOU()
            large_rect()

"""
class RandomResizeLong():
    """Resizes images from dataset using PIL to random shape between min_long and max_long"""

    def __init__(self, min_long, max_long):
        self.min_long = min_long
        self.max_long = max_long

    def __call__(self, img, sal=None):
        target_long = random.randint(self.min_long, self.max_long)
        w, h = img.size

        if w < h:
            target_shape = (int(round(w * target_long / h)), target_long)
        else:
            target_shape = (target_long, int(round(h * target_long / w)))

        img = img.resize(target_shape, resample=PIL.Image.CUBIC)
        if sal:
           sal = sal.resize(target_shape, resample=PIL.Image.CUBIC)
           return img, sal
        return img


class RandomCrop():
    """Randomly crops images"""
    def __init__(self, cropsize):
        self.cropsize = cropsize

    def __call__(self, imgarr, sal=None):
        print(np.shape(imgarr) )
        # h, w = np.shape(imgarr)                  # height, width
        w, h = img.size
        ch = min(self.cropsize, h)
        cw = min(self.cropsize, w)

        w_space = w - self.cropsize
        h_space = h - self.cropsize

        if w_space > 0:
            cont_left = 0
            img_left = random.randrange(w_space+1)
        else:
            cont_left = random.randrange(-w_space+1)
            img_left = 0

        if h_space > 0:
            cont_top = 0
            img_top = random.randrange(h_space+1)
        else:
            cont_top = random.randrange(-h_space+1)
            img_top = 0

        container = np.zeros((self.cropsize, self.cropsize, np.shape(imgarr)[-1]), np.float32)
        container[cont_top:cont_top+ch, cont_left:cont_left+cw] = \
            imgarr[img_top:img_top+ch, img_left:img_left+cw]
        if sal is not None:
            container_sal = np.zeros((self.cropsize, self.cropsize,1), np.float32)
            container_sal[cont_top:cont_top+ch, cont_left:cont_left+cw,0] = \
                sal[img_top:img_top+ch, img_left:img_left+cw]
            return container, container_sal

        return container

def get_random_crop_box(imgsize, cropsize):
    h, w = imgsize

    ch = min(cropsize, h)
    cw = min(cropsize, w)

    w_space = w - cropsize
    h_space = h - cropsize

    if w_space > 0:
        cont_left = 0
        img_left = random.randrange(w_space + 1)
    else:
        cont_left = random.randrange(-w_space + 1)
        img_left = 0

    if h_space > 0:
        cont_top = 0
        img_top = random.randrange(h_space + 1)
    else:
        cont_top = random.randrange(-h_space + 1)
        img_top = 0

    return cont_top, cont_top+ch, cont_left, cont_left+cw, img_top, img_top+ch, img_left, img_left+cw

def crop_with_box(img, box):
    if len(img.shape) == 3:
        img_cont = np.zeros((max(box[1]-box[0], box[4]-box[5]), max(box[3]-box[2], box[7]-box[6]), img.shape[-1]), dtype=img.dtype)
    else:
        img_cont = np.zeros((max(box[1] - box[0], box[4] - box[5]), max(box[3] - box[2], box[7] - box[6])), dtype=img.dtype)
    img_cont[box[0]:box[1], box[2]:box[3]] = img[box[4]:box[5], box[6]:box[7]]
    return img_cont


def random_crop(images, cropsize, fills):
    if isinstance(images[0], PIL.Image.Image):
        imgsize = images[0].size[::-1]
    else:
        imgsize = images[0].shape[:2]
    box = get_random_crop_box(imgsize, cropsize)

    new_images = []
    for img, f in zip(images, fills):

        if isinstance(img, PIL.Image.Image):
            img = img.crop((box[6], box[4], box[7], box[5]))
            cont = PIL.Image.new(img.mode, (cropsize, cropsize))
            cont.paste(img, (box[2], box[0]))
            new_images.append(cont)

        else:
            if len(img.shape) == 3:
                cont = np.ones((cropsize, cropsize, img.shape[2]), img.dtype)*f
            else:
                cont = np.ones((cropsize, cropsize), img.dtype)*f
            cont[box[0]:box[1], box[2]:box[3]] = img[box[4]:box[5], box[6]:box[7]]
            new_images.append(cont)

    return new_images


class AvgPool2d():
    """Average pooling on 2d"""
    def __init__(self, ksize):
        self.ksize = ksize

    def __call__(self, img):
        import skimage.measure

        return skimage.measure.block_reduce(img, (self.ksize, self.ksize, 1), np.mean)


class RandomHorizontalFlip():
    """Randomly mirror images"""
    def __init__(self):
        return

    def __call__(self, img, sal=None):
        if bool(random.getrandbits(1)):
            #img = img.transpose(PIL.Image.FLIP_LEFT_RIGHT)
            img = np.fliplr(img).copy()
            if sal:
                #sal = sal.transpose(PIL.Image.FLIP_LEFT_RIGHT)
                sal = np.fliplr(sal).copy()
                return img, sal 
            return img
        else:
            if sal:
                return img, sal
            return img


class CenterCrop():
    """Crop images centered"""
    def __init__(self, cropsize, default_value=0):
        self.cropsize = cropsize
        self.default_value = default_value

    def __call__(self, npimg):
        h, w = npimg.shape[:2]

        ch = min(self.cropsize, h)
        cw = min(self.cropsize, w)

        sh = h - self.cropsize
        sw = w - self.cropsize

        if sw > 0:
            cont_left = 0
            img_left = int(round(sw / 2))
        else:
            cont_left = int(round(-sw / 2))
            img_left = 0

        if sh > 0:
            cont_top = 0
            img_top = int(round(sh / 2))
        else:
            cont_top = int(round(-sh / 2))
            img_top = 0

        if len(npimg.shape) == 2:
            container = np.ones((self.cropsize, self.cropsize), npimg.dtype)*self.default_value
        else:
            container = np.ones((self.cropsize, self.cropsize, npimg.shape[2]), npimg.dtype)*self.default_value

        container[cont_top:cont_top+ch, cont_left:cont_left+cw] = \
            npimg[img_top:img_top+ch, img_left:img_left+cw]

        return container


def HWC_to_CHW(tensor, sal=False):
    if sal:
        tensor = np.expand_dims(tensor, axis=0)
    else:
        tensor = np.transpose(tensor, (2, 0, 1))
    return tensor


class RescaleNearest():
    """Resize image using nearest neighmor interpolation"""
    def __init__(self, scale):
        self.scale = scale

    def __call__(self, npimg):
        import cv2
        return cv2.resize(npimg, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)


def bb_IOU(boxA, boxB):
    """COmputes IOU based on bounding boxes (predicted and ground truth)"""
    boxA = [float(aa) for aa in boxA]
    boxB = [float(bb) for bb in boxB]

    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    if xA >= xB or yA >= yB:
        return 0, 0
    # compute the area of intersection rectangle
    interArea = (xB - xA + 1) * (yB - yA + 1)

    # compute the area of both the prediction and ground-truth rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    recall = interArea / float(boxAArea)
    return iou, recall

def large_rect(rect):
    """find largest rectangles"""
    large_area = 0
    target = 0
    for i in range(len(rect)):
        area = rect[i][2]*rect[i][3]
        if large_area < area:
            large_area = area
            target = i

    x = rect[target][0]
    y = rect[target][1]
    w = rect[target][2]
    h = rect[target][3]

    return x, y, w, h

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    
def crf_inference(img, probs, t=10, scale_factor=1, labels=21):
    import pydensecrf.densecrf as dcrf
    from pydensecrf.utils import unary_from_softmax

    h, w = img.shape[:2]
    n_labels = labels

    d = dcrf.DenseCRF2D(w, h, n_labels)

    unary = unary_from_softmax(probs)
    unary = np.ascontiguousarray(unary)
    unary = unary.reshape((n_labels,-1))
    d.setUnaryEnergy(unary)
    d.addPairwiseGaussian(sxy=3/scale_factor, compat=3)
    d.addPairwiseBilateral(sxy=80/scale_factor, srgb=13, rgbim=np.copy(img), compat=10)
    Q = d.inference(t)

    return np.array(Q).reshape((n_labels, h, w))


# train.py

In [ ]:
model = getattr(importlib.import_module(network), 'Net')(k_cluster, from_round_nb)
model.load_state_dict(torch.load(weights))

# model.eval()
model.cuda()

k_cluster: 10
Round: 0


Net(
  (conv1a): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (b2): ResBlock(
    (bn_branch2a): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_branch2a): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn_branch2b1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_branch2b1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (conv_branch1): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
  )
  (b2_1): ResBlock(
    (bn_branch2a): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_branch2a): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn_branch2b1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_branch2b1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), pad

In [45]:
weight

'./pretrained_model/res38_cls.pth'

In [ ]:
#  model.fc8 = torch.nn.Conv2d(4096, 8, 1, bias=False)
#  model.fc8

Conv2d(4096, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)

In [ ]:
# infer_dataset = voc12.data.VOC12ClsDatasetMSF(infer_list, data_root=data_root,
#                                                scales=(1, 0.5, 1.5, 2.0),
#                                                  inter_transform=torchvision.transforms.Compose(
#                                                    [np.asarray,
#                                                     model.normalize,
#                                                     HWC_to_CHW
#                                                     ]))

# infer_data_loader = DataLoader(infer_dataset, shuffle=False, num_workers=num_workers, pin_memory=True)


In [ ]:
# for iter, (img_name, img_list, label) in enumerate(infer_data_loader):
#         # img_name = img_name[0]
#         break

In [ ]:
# img_name, img_list[0].shape, label

In [19]:
colormaps = ['#000000', '#996633', '#ff9900', '#cc0066', '#660066', '#3333cc', '#00ffff', '#33cc33', '#ffff00']

def colormap(index):
    return mpl.colors.LinearSegmentedColormap.from_list('cmap', [colormaps[0], colormaps[index+1], '#FFFFFF'], 256)

In [ ]:
from torchvision.datasets import ImageFolder
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset = ImageFolder(root=data_root)
class_names = dataset.classes
num_classes = len(class_names)
train_len = int(0.9 * len(dataset))
val_len = len(dataset) - train_len
train_set, val_set = torch.utils.data.random_split(dataset, [train_len, val_len])

transform = {
    "train": transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(256),
            transforms.Resize(224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.RandomRotation(90),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
        ]
    ),
    "test": transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(256),
            transforms.Resize(224),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
        ]
    ),
}

train_set = MyLazyDataset(train_set, transform["train"])
val_set = MyLazyDataset(val_set, transform["test"])

# train_data_loader = DataLoader(
#     dataset=train_set, batch_size=batch_size, shuffle=True, num_workers=2
# )
# val_data_loader = DataLoader(
#     dataset=val_set, batch_size=batch_size, shuffle=False, num_workers=2
# )

In [ ]:
data = ImageFolderWithPaths(root=train_set, transform=transform['train'], round_nb=from_round_nb)
train_data_loader = torch.utils.data.DataLoader(dataset=data, batch_size=batch_size, shuffle=True, num_workers=2)

data = ImageFolderWithPaths(root=val_set, transform=transform['test'], round_nb=from_round_nb)
val_data_loader = torch.utils.data.DataLoader(dataset=data, batch_size=batch_size, shuffle=False, num_workers=2)

In [ ]:
class_names

In [ ]:
torch.multiprocessing.freeze_support()
test_dir = '/content/drive/My Drive/MAI/thesis/source/kvasir-dataset-v2-folds/1/test'
train_dir = '/content/drive/My Drive/MAI/thesis/source/kvasir-dataset-v2-folds/1/train'

mapp, test_data_loader = get_data_loader(data_dir=test_dir, batch_size=batch_size, train=False)
mapp, train_data_loader = get_data_loader(data_dir=train_dir, batch_size=batch_size, train=True)

In [ ]:
# fname_new = fname[0].split('/')
# im_name = fname_new[-1][0:-4]
# im_name

In [ ]:
# imgs.shape, fname

In [ ]:
len(train_data_loader)

900

In [ ]:
 for iter, dat in enumerate(train_data_loader, 0):
   print(len(dat))
   break

5


In [ ]:
dat[0].shape, dat[1].shape

(torch.Size([8, 3, 224, 224]), torch.Size([8]))

In [ ]:
import numpy as np
if True:
  n_gpus = torch.cuda.device_count()
  # model_replicas = torch.nn.parallel.replicate(model, list(range(n_gpus)))
  image_feature_arr = np.empty([0,4096])

  print('################################ Extracting features from Round-{} ...... ################################'.format(from_round_nb))
  for iter, (imgs, lbls, path, _, _) in enumerate(train_data_loader, 0):
      if from_round_nb == 0:
          tmp, feature, _ = model.forward(imgs.cuda(), from_round_nb)
      else:
          tmp, feature, y_8, x_80, y_80 = model.forward(imgs.cuda(), from_round_nb)
      feature = feature.cpu().detach().numpy()
      image_feature_arr = np.concatenate((image_feature_arr, feature), 0)

      if iter % 100 == 0:
          print('Already extracted: {}/{}'.format(iter, len(train_data_loader)))
          # print(image_feature_arr.shape)


  # image_feature_arr = np.array(image_feature_arr)
  print(image_feature_arr.shape)

  # save the extracted feature
  save_feature_folder_path = os.path.join(save_folder, 'feature')
  feature_save_path = os.path.join(save_feature_folder_path, 'R{}_feature.npy'.format(from_round_nb)) # R1 feature is for R2 use
  np.save(feature_save_path, image_feature_arr)
  print('extract_feature.py done')

################################ Extracting features from Round-0 ...... ################################
Already extracted: 0/900
Already extracted: 100/900
Already extracted: 200/900
Already extracted: 300/900
Already extracted: 400/900
Already extracted: 500/900
Already extracted: 600/900
Already extracted: 700/900
Already extracted: 800/900
(7200, 4096)
extract_feature.py done


In [21]:
import sys
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AffinityPropagation
from sklearn.manifold import TSNE

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.patches import Rectangle

import scipy.misc as sim
from PIL import Image
import pandas as pd
import shutil
import os
import argparse

In [22]:
def read_train_list():
    with open('./voc12/train_aug.txt', 'r') as r:
        filename_list = r.read().split('\n')
    r.close()

    filename_list = filename_list[:-1]
    for num, filename in enumerate(filename_list):
        filename_list[num] = filename[12:23]
    return filename_list
    
def read_train_list_new(data_loader):
    fname_list = []
    lbls_list = []
    for iter, (_, lbls, fname, _,_) in enumerate(data_loader, 0):
          for im in range(len(fname)):
            fname_new = fname[im].split('/')
            fname_list.append(fname_new[-1][0:-4])
            lbls_list.append(F.one_hot(lbls[im], num_classes=8).numpy())
            # print(fname_new[-1][0:-4])
    
    return fname_list, np.stack(lbls_list)


def make_subclass_label(class_idx, labels, subclass_nb, all_class_kmeans_label):
    final_label = []

    for i in range(len(labels)):
        label = [0]*subclass_nb*8
        label[class_idx*subclass_nb+labels[i]] = 1
        all_class_kmeans_label.append(label)

    return all_class_kmeans_label


def generate_merged_label(repeat_list, label_list):
    new_label_list = []
    for num, one in enumerate(repeat_list):
        merged_label = []
        for i in one:
            merged_label.append(label_list[i])
        merged_label = sum(np.array(merged_label))  # merge the kmeans label to make the sub-category label

        nb_subcategory = np.nonzero(merged_label)[0].shape[0]  # check the number of sub-category
        if len(one) == nb_subcategory:
            new_label_list.append(merged_label)

    return new_label_list


def create_class_key_in_dict(dict, cls_nb):
    for i in range(cls_nb):
        dict[i] = []

    return dict


def make_filename_class_dict(filenamelist, labels):
    filename_idx_class_dict = {}
    filename_idx_class_dict = create_class_key_in_dict(filename_idx_class_dict, 8)

    filename_class_dict = {}
    filename_class_dict = create_class_key_in_dict(filename_class_dict, 8)

    for num, one in enumerate(labels):
        gt_labels = np.where(one == 1)[0]
        for gt in gt_labels:
            filename_idx_class_dict[gt].append(num)
            filename_class_dict[gt].append(filenamelist[num])

    return filename_idx_class_dict, filename_class_dict


def merge_filename_class_dict(filename_class_dict):
    merged_filename_list = []
    for i in range(8):
        class_filename_list = filename_class_dict[i]
        for one in class_filename_list:
            merged_filename_list.append(one)

    return merged_filename_list


def generate_repeat_list(filename_list):
    repeat_list = []
    for num, one in enumerate(filename_list):
        repeat_idx_list = []
        for num_, one_ in enumerate(filename_list):
            if one == one_:
                repeat_idx_list.append(num_)
        repeat_list.append(repeat_idx_list)

    return repeat_list


def remove_duplicate_label(repeat_list):
    keep_idx_list = []
    remove_idx_list = []
    for repeat_set in repeat_list:
        for num, one in enumerate(repeat_set):
            if num == 0 and one not in keep_idx_list:
                keep_idx_list.append(one)
            else:
                remove_idx_list.append(one)

    return keep_idx_list, remove_idx_list



def create_train_data(merge_filename_list, new_label_list, keep_idx_list, label_20):
    # label_20 = np.load('./voc12/20_class_labels.npy')
    print(len(merge_filename_list), len(new_label_list), label_20.shape)

    train_filename_list = []
    train_label_200 = []
    train_label_20 = []
    for idx in keep_idx_list:
        train_filename_list.append(merge_filename_list[idx])
        train_label_200.append(new_label_list[idx])
        train_label_20.append(label_20[idx])

        tmp = np.where(new_label_list[idx] == 1)[0]
        cls200_par = [int(x/10) for x in tmp]

    return train_filename_list, train_label_200, train_label_20

Create pseudo-labels


In [ ]:
labels_list = []
for iter, (_, lbls, path, _, _) in enumerate(train_data_loader, 0):
  for im in range(len(lbls)):
    labels_list.append(F.one_hot(lbls[im], num_classes=8))
  break

k = torch.stack(labels_list)
k

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff8502248c0>
Traceback (most recent call last):
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff8502248c0>
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
Traceback (most recent call last):
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
    self._shutdown_workers()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
  File "/usr/lib/pytho

tensor([[0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1]])

In [ ]:
voc_list = read_train_list()

In [ ]:
kvasir_list, labels_list = read_train_list_new(train_data_loader)

In [ ]:
filename_idx_class_dict, filename_class_dict = make_filename_class_dict(kvasir_list, labels_list)

In [ ]:
k_cluster = 10
for_round_nb = 1
feature_folder_path = os.path.join(save_folder, 'feature')

In [ ]:
merge_filename_list = merge_filename_class_dict(filename_class_dict)
repeat_list = generate_repeat_list(merge_filename_list)
keep_idx_list, remove_idx_list = remove_duplicate_label(repeat_list)

In [ ]:
if for_round_nb == 1:
    features = np.load('{}/R{}_feature.npy'.format(feature_folder_path, for_round_nb-1))
else:
    features = np.load('{}/R{}_feature.npy'.format(feature_folder_path, for_round_nb-1))

In [ ]:
all_class_kmeans_label = []
for i in range(8):
    filename_idx_list = filename_idx_class_dict[i]

    class_feature_list = []
    for idx in filename_idx_list:
        class_feature_list.append(features[idx])
    print('Class {}: {}'.format(i, len(class_feature_list)))

    # apply kmeans
    X = class_feature_list
    k_cluster = k_cluster
    max_iter = 300

    k_center = KMeans(n_clusters=k_cluster, random_state=0, max_iter=max_iter).fit(X)
    labels = k_center.labels_
    centers = k_center.cluster_centers_
    iter_nb = k_center.n_iter_
    distance = k_center.inertia_
    subclass_nb = k_cluster

    # generate the sub-category label
    all_class_kmeans_label = make_subclass_label(i, labels, subclass_nb, all_class_kmeans_label)


Class 0: 900
Class 1: 900
Class 2: 900
Class 3: 900
Class 4: 900
Class 5: 900
Class 6: 900
Class 7: 900


In [ ]:
# merge the kmeans label to make the sub-category label
new_label_list = generate_merged_label(repeat_list, all_class_kmeans_label)
print(len(merge_filename_list), len(new_label_list))   # 16458


# create the parent label and the sub-category label as the training data with
train_filename_list, train_label_200, train_label_20 = create_train_data(merge_filename_list, new_label_list, keep_idx_list, labels_list)
print(len(train_filename_list), len(train_label_200), len(train_label_20))

train_label_200 = np.array(train_label_200)
train_label_20 = np.array(train_label_20)
print(train_label_200.shape, train_label_20.shape)    # VOC:(10582, 200) (10582, 20), KVASIR: (7200, 80) (7200, 8)


7200 7200
7200 7200 (7200, 8)
7200 7200 7200
(7200, 80) (7200, 8)


In [ ]:
train_label_200.shape, train_label_20.shape    # VOC:(10582, 200) (10582, 20), KVASIR: (7200, 80) (7200, 8)

((7200, 80), (7200, 8))

In [ ]:
with open('{}/label/R{}_train_filename_list.txt'.format(save_folder, for_round_nb), 'w') as f:
    for x in train_filename_list:
        line = '{}\n'.format(x)
        f.write(line)
f.close()

np.save('{}/label/R{}_train_label_200.npy'.format(save_folder, for_round_nb), train_label_200)
np.save('{}/label/R{}_train_label_20.npy'.format(save_folder, for_round_nb), train_label_20)

print('##################### k_{} Round-{} pseudo labels are saved at {}/label. #####################'.format(k_cluster, for_round_nb, save_folder))
print('##################### You can start to train the classification model. #####################')


##################### k_10 Round-1 pseudo labels are saved at ./results_kvasir/label. #####################
##################### You can start to train the classification model. #####################


# Train classes

In [24]:
def save_checkpoint(state, filename='checkpoint.pth.tar'):     # save points during training
    """Function for saving checkpoints"""
    savepath = os.path.join(snapshot_dir, filename)
    torch.save(state, savepath)
    print(f"Model saved to {savepath}")
   

def load_ckp(checkpoint_fpath, model, optimizer):
    if not torch.cuda.is_available():
      checkpoint = torch.load(checkpoint_fpath, map_location=torch.device('cpu'))
    else:
      checkpoint = torch.load(checkpoint_fpath)

    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in checkpoint['state_dict'].items():
        name = k[7:]                                  # remove `module.`
        new_state_dict[name] = v
    
    model.load_state_dict(new_state_dict)
    optimizer.load_state_dict(checkpoint['optimizer'])
    print('weights loaded!')
    return model, optimizer, checkpoint['epoch']+1, checkpoint['global_counter']

def get_model():
    """Function for getting VGG16"""
    model = vgg.vgg16(pretrained=True, num_classes=num_classes, att_dir=att_dir, training_epoch=epoch)
    model = torch.nn.DataParallel(model).cuda()                         # parallel GPU training
    param_groups = model.module.get_parameter_groups()

    optimizer = optim.Adam([{'params': param_groups[0], 'lr': lr},
        {'params': param_groups[1], 'lr': 4*lr},
        {'params': param_groups[2], 'lr': 15*lr},
        {'params': param_groups[3], 'lr': 30*lr}]
        , lr=lr,  weight_decay=weight_decay)
    return  model, optimizer

def prepare_model():
    """Taken from hyper-Kvasir classification"""
    model = models.resnet152(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 8)
    model = model.to(device)
    
    return model

def prepare_model_custom():
    """Function for getting ResNet152"""
    model = resnet152(num_classes=num_classes, att_dir=att_dir, training_epoch=epoch)
    model = torch.nn.DataParallel(model).cuda()                         # parallel GPU training
    param_groups = model.module.get_parameter_groups()

    optimizer = optim.Adam([{'params': param_groups[0], 'lr': lr},
        {'params': param_groups[1], 'lr': 4*lr},
        {'params': param_groups[2], 'lr': 15*lr},
        {'params': param_groups[3], 'lr': 30*lr}]
        , lr=lr,  weight_decay=weight_decay)
    return  model, optimizer

In [25]:
def compute_acc(pred_labels, gt_labels):
    pred_correct_count = 0
    pred_correct_list = []
    for pred_label in pred_labels:
        if pred_label in gt_labels:
            pred_correct_count += 1
    union = len(gt_labels) + len(pred_labels) - pred_correct_count
    acc = round(pred_correct_count/union, 4)
    return acc


def cls200_sum(y_200):
    cls20_prob_sum_list = []
    for rou in range(8):
        subclass_prob_sum = sum(y_200[rou*k_cluster:rou*k_cluster+k_cluster])
        cls20_prob_sum_list.append(subclass_prob_sum/10)

    # cls200_pred_max = np.argmax(cls20_prob_sum_list)
    cls200_pred_max = np.zeros_like(y_200)
    cls200_pred_max[np.argmax(y_200)] = 1
    cls200_pred_max = np.where(np.array(cls20_prob_sum_list)>0.05)[0]
    return cls200_pred_max


def get_img_path(img_name, dataset_path):
    tmp = os.path.join(dataset_path, img_name + '.jpg')
    return tmp

# Train

In [66]:
batch_size = 4
max_epoches = 31
network = "network.resnet38_cls_kvasir"
lr = 0.01
num_workers = 2
wt_dec = 5e-4

resume = True
weights = './pretrained_model/ilsvrc_cls.params'
weights = './pretrained_model/ilsvrc-cls_rna-a1_cls1000_ep-0001.params'
session_name = "resnet_cls"
crop_size = 224
subcls_loss_weight = 5
round_nb = 1
k_cluster = 10
disp_interval = 100
snapshot_dir = 'checkpoints/train/exp2/'        # where to save models

In [27]:
torch.multiprocessing.freeze_support()
test_dir = '/content/drive/My Drive/MAI/thesis/source/kvasir-dataset-v2-folds/1/test'
train_dir = '/content/drive/My Drive/MAI/thesis/source/kvasir-dataset-v2-folds/1/train'

mapp, test_data_loader = get_data_loader(data_dir=test_dir, batch_size=batch_size, train=False)
mapp, train_data_loader = get_data_loader(data_dir=train_dir, batch_size=batch_size, train=True)

In [ ]:
def validate(model, val_loader, loss_fn):
    """Function for validating"""
    # print('\nvalidating ... \n', flush=True, end='')
    val_loss = AverageMeter()

    model.eval()
    running_corrects = 0
    with torch.no_grad():
        for idx, dat in enumerate(val_loader):
            img_name, img, label = dat
            # print('img size ', img.size())
            img = img.cuda()
            label = label.cuda()
            logits = model(img)
            _, preds = torch.max(logits, 1)
            # print('logits shape:', logits.shape)
            if len(logits.shape) == 1:
                logits = logits.reshape(label.shape)
            print(logits)
            loss_val = loss_fn(logits, torch.max(label, 1)[1].long())         # find loss  
            val_loss.update(loss_val.data.item(), img.size()[0])              # update val loss with average of batch_images


    running_corrects += torch.sum(preds == torch.max(label, 1)[1])
    val_acc = running_corrects.double() / len(val_dataset)
    neptune.log_metric('val_loss', val_loss.val)
    neptune.log_metric('val_acc', val_acc)
    print('validation loss:', val_loss.val, 'accuracy: ', val_acc)

    return val_loss

In [28]:
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from torchvision import datasets, models

def worker_init_fn(worker_id):
        np.random.seed(1 + worker_id)

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [39]:
label_20 = np.load('{}/label/R{}_train_label_20.npy'.format(save_folder, round_nb)) # 16458
label_200 = np.load('{}/label/R{}_train_label_200.npy'.format(save_folder, round_nb))

In [ ]:
label_20.shape, label_200.shape

((7200, 8), (7200, 80))

In [40]:
model = getattr(importlib.import_module(network), 'Net')(k_cluster, round_nb)
pyutils.Logger(session_name + '.log')

log_path = os.path.join(save_folder, 'log', 'R{}'.format(round_nb))

In [41]:
from utils import pyutils, imutils, torchutils

global_counter = 0
best_val_loss = float('inf')
current_epoch = 0
epochs = max_epoches
dataset = 'kvasir'


print('Training started ...')
PARAMS = {'dataset':dataset,
            'network':'resnet38',
            'epoch_nr': epochs,
            'batch_size': batch_size,
            'optimizer': 'PolySGD',
            'lr': lr
      }
neptune.create_experiment('kvasirv2_train', params=PARAMS)
max_step = len(train_data_loader)  * epochs


param_groups = model.get_parameter_groups()
optimizer = torchutils.PolyOptimizer([
    {'params': param_groups[0], 'lr': lr, 'weight_decay': wt_dec},
    {'params': param_groups[1], 'lr': 2*lr, 'weight_decay': 0},
    {'params': param_groups[2], 'lr': 10*lr, 'weight_decay': wt_dec},
    {'params': param_groups[3], 'lr': 20*lr, 'weight_decay': 0}
], lr=lr, weight_decay=wt_dec, max_step=max_step)

In [36]:
device

device(type='cpu')

In [ ]:
# model, optimizer, current_epoch, global_counter = load_ckp('/content/drive/MyDrive/MAI/thesis/source/SC-CAM/checkpoints/train/exp2/kvasir_epoch_10.pth', model, optimizer)

In [46]:
if resume:
  model, optimizer, current_epoch, global_counter = load_ckp('/content/drive/MyDrive/MAI/thesis/source/SC-CAM/checkpoints/train/exp2/kvasir_epoch_10.pth', model, optimizer)
else:
  if weights[-7:] == '.params':
      # assert network == "network.resnet38_cls"
      import network.resnet38d
      weights_dict = network.resnet38d.convert_mxnet_to_torch(weights)
  elif weights[-11:] == '.caffemodel':
      # assert network == "network.vgg16_cls"
      import network.vgg16d
      weights_dict = network.vgg16d.convert_caffe_to_torch(weights)
  else:
      weights_dict = torch.load(weights)

  model.load_state_dict(weights_dict, strict=False)
# model = torch.nn.DataParallel(model).cuda()
model.train()

avg_meter = pyutils.AverageMeter('loss')

# parent_labels = np.load('./voc12/cls_labels.npy', allow_pickle=True).tolist()

In [47]:
model

Net(
  (conv1a): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (b2): ResBlock(
    (bn_branch2a): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_branch2a): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn_branch2b1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_branch2b1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (conv_branch1): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
  )
  (b2_1): ResBlock(
    (bn_branch2a): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_branch2a): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn_branch2b1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_branch2b1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), pad

In [51]:
weights_dict = torch.load('/content/drive/MyDrive/MAI/thesis/source/SC-CAM/checkpoints/train/exp2/kvasir_epoch_10.pth', map_location=torch.device('cpu'))

In [56]:
model.state_dict().keys()

odict_keys(['conv1a.weight', 'b2.bn_branch2a.weight', 'b2.bn_branch2a.bias', 'b2.bn_branch2a.running_mean', 'b2.bn_branch2a.running_var', 'b2.bn_branch2a.num_batches_tracked', 'b2.conv_branch2a.weight', 'b2.bn_branch2b1.weight', 'b2.bn_branch2b1.bias', 'b2.bn_branch2b1.running_mean', 'b2.bn_branch2b1.running_var', 'b2.bn_branch2b1.num_batches_tracked', 'b2.conv_branch2b1.weight', 'b2.conv_branch1.weight', 'b2_1.bn_branch2a.weight', 'b2_1.bn_branch2a.bias', 'b2_1.bn_branch2a.running_mean', 'b2_1.bn_branch2a.running_var', 'b2_1.bn_branch2a.num_batches_tracked', 'b2_1.conv_branch2a.weight', 'b2_1.bn_branch2b1.weight', 'b2_1.bn_branch2b1.bias', 'b2_1.bn_branch2b1.running_mean', 'b2_1.bn_branch2b1.running_var', 'b2_1.bn_branch2b1.num_batches_tracked', 'b2_1.conv_branch2b1.weight', 'b2_2.bn_branch2a.weight', 'b2_2.bn_branch2a.bias', 'b2_2.bn_branch2a.running_mean', 'b2_2.bn_branch2a.running_var', 'b2_2.bn_branch2a.num_batches_tracked', 'b2_2.conv_branch2a.weight', 'b2_2.bn_branch2b1.weight',

In [62]:
optimizer.state_dict()['param_groups']

[{'dampening': 0,
  'lr': 0.0067408137685441355,
  'momentum': 0.0005,
  'nesterov': False,
  'params': [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42],
  'weight_decay': 0.0005},
 {'dampening': 0,
  'lr': 0.013481627537088271,
  'momentum': 0.0005,
  'nesterov': False,
  'params': [],
  'weight_decay': 0},
 {'dampening': 0,
  'lr': 0.06740813768544135,
  'momentum': 0.0005,
  'nesterov': False,
  'params': [43, 44],
  'weight_decay': 0.0005},
 {'dampening': 0,
  'lr': 0.1348162753708827,
  'momentum': 0.0005,
  'nesterov': False,
  'params': [],
  'weight_decay': 0}]

In [57]:
optimizer.param_groups[0]['lr']

0.0067408137685441355

In [ ]:
# for num, one in enumerate(cls20_prob):
#     ep_count += 1
#     pass_cls = np.zeros_like(el)
#     pass_cls[np.argmax(el)] = 1
#     true_cls_20 = F.one_hot(label_20[num], num_classes=8)

In [ ]:
# true_cls_20

tensor([0, 0, 0, 0, 1, 0, 0, 0], device='cuda:0')

In [ ]:
# F.one_hot(label_20, num_classes=8)

tensor([[0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0]], device='cuda:0')

In [ ]:
# x_20, y_20

(tensor([[-0.4334, -0.7389, -0.7698, -0.0764,  0.7463, -0.3573,  0.2985,  0.6670],
         [-0.9975,  0.0150, -0.2557, -0.1252,  0.3104,  0.2030,  0.2711,  0.3772],
         [ 0.8922,  0.1680, -0.3851,  0.9093, -0.1707, -0.4461,  0.3407,  0.0655],
         [ 0.9441, -0.3392,  0.4351,  1.1866,  1.3708,  0.1402,  0.6759, -0.2483]],
        device='cuda:0', grad_fn=<ViewBackward>),
 tensor([[0.3933, 0.3233, 0.3165, 0.4809, 0.6784, 0.4116, 0.5741, 0.6608],
         [0.2694, 0.5037, 0.4364, 0.4687, 0.5770, 0.5506, 0.5674, 0.5932],
         [0.7093, 0.5419, 0.4049, 0.7129, 0.4574, 0.3903, 0.5844, 0.5164],
         [0.7199, 0.4160, 0.6071, 0.7661, 0.7975, 0.5350, 0.6628, 0.4382]],
        device='cuda:0', grad_fn=<SigmoidBackward>))

In [ ]:
# for num, el in enumerate(tmp):
#             pass_cls = np.where(el>0.5)[0]

In [ ]:
# tmp.shape, tmp_label.shape, np.argmax(pass_cls), np.argmax()

((4, 80), (4, 80), 39)

In [ ]:
        # tmp = y_200.cpu().data.numpy()
        # tmp_label = label_200.cpu().data.numpy()
        # for num, el in enumerate(tmp):
        #     pass_cls = np.where(el>0.5)[0]
        #     true_cls_200 = np.where(tmp_label[num] == 1)[0]

        #     parent_cls = label_20[num].cpu().numpy()
        #     cls200_pred_max = cls200_sum(el)

In [ ]:
# parent_cls.item()

4

In [ ]:
    # cls20_prob_sum_list = []
    # for rou in range(8):
    #     subclass_prob_sum = sum(el[rou*k_cluster:rou*k_cluster+k_cluster])
    #     cls20_prob_sum_list.append(subclass_prob_sum/10)

In [ ]:
# cls20_prob_sum_list, sum(el[rou*k_cluster:rou*k_cluster+k_cluster])

([0.4183946892619133,
  0.4592417746782303,
  0.563051524758339,
  0.5149950951337814,
  0.4251436829566956,
  0.6100198566913605,
  0.5427002787590027,
  0.4869419902563095],
 4.235070139169693)

In [ ]:
# label_20.cpu().numpy()

array([1, 3, 1, 6])

In [ ]:
        # tmp = y_200.cpu().data.numpy()
        # tmp_label = label_200.cpu().data.numpy()
        # for num, el in enumerate(tmp):
        #     pass_cls = np.where(el>0.5)[0]
        #     true_cls_200 = np.where(tmp_label[num] == 1)[0]

        #     parent_cls = F.one_hot(label_20[num], num_classes=8)
        #     cls200_pred_max = cls200_sum(el)

        #     if np.array_equal(pass_cls, true_cls_200) == True:
        #         ep_EM += 1

        #     # cls200: acc for 200-->20 (sum)
        #     acc = compute_acc(cls200_pred_max, parent_cls)

In [ ]:
# cls200_pred_max, parent_cls, acc

(array([0, 1, 2, 3, 4, 5, 6, 7]),
 tensor([0, 0, 0, 0, 0, 0, 1, 0], device='cuda:0'),
 0.1429)

In [67]:
while current_epoch < epochs:  # loop over the dataset multiple times
    total_loss = 0.0
    corrects = 0
    total_samples = 0

    ep_count = 0
    ep_EM = 0
    ep_acc = 0

    ep_p_EM = 0
    ep_p_acc = 0
    ep_acc_vote = 0

    cls20_ep_EM = 0
    cls20_ep_acc = 0
    for iter, dat in enumerate(train_data_loader,0):
        inputs, label_20, fname, l, label_200 = dat
        inputs = inputs.to(device)
        label_20 = label_20.to(device)
        label_200 = label_200.to(device)
        img_name = fname

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        x_20, _, y_20, x_200, y_200 = model(inputs, round_nb)
        cls20_prob = y_20.cpu().data.numpy()
        cls20_gt = label_20.cpu().data.numpy()

        for num, el in enumerate(cls20_prob):
            ep_count += 1
            pass_cls = np.zeros_like(el)
            pass_cls[np.argmax(el)] = 1
            true_cls_20 = F.one_hot(label_20[num], num_classes=8)

            if np.array_equal(pass_cls, true_cls_20) == True:  # exact match
                cls20_ep_EM += 1

            acc = compute_acc(pass_cls, true_cls_20)
            cls20_ep_acc += acc

        # compute acc for 200 classes
        tmp = y_200.cpu().data.numpy()
        tmp_label = label_200.cpu().data.numpy()
        for num, el in enumerate(tmp):
            pass_cls = np.where(el>0.5)[0]
            true_cls_200 = np.where(tmp_label[num] == 1)[0]

            parent_cls = F.one_hot(label_20[num], num_classes=8)
            cls200_pred_max = cls200_sum(el)

            if np.array_equal(pass_cls, true_cls_200) == True:
                ep_EM += 1

            # cls200: acc for 200-->20 (sum)
            acc = compute_acc(cls200_pred_max, parent_cls)
            ep_acc += acc

            # cls200: acc for 200-->20 (top 1)
            pass_map_cls = np.unique([int(m/k_cluster) for m in np.where(el > 0.5)[0]])
            if np.array_equal(pass_map_cls, parent_cls) == True:
                ep_p_EM += 1

            p_acc = compute_acc(pass_map_cls, parent_cls)
            ep_p_acc += p_acc

        avg_ep_EM = round(ep_EM/ep_count, 4)
        avg_ep_acc = round(ep_acc/ep_count, 4)

        avg_ep_p_EM = round(ep_p_EM/ep_count, 4)
        avg_ep_p_acc = round(ep_p_acc/ep_count, 4)

        avg_cls20_ep_EM = round(cls20_ep_EM/ep_count, 4)
        avg_cls20_ep_acc = round(cls20_ep_acc/ep_count, 4)

        cls_20_loss = F.multilabel_soft_margin_loss(x_20, F.one_hot(label_20, num_classes=8))
        cls_200_loss = F.multilabel_soft_margin_loss(x_200, label_200)

        loss = cls_20_loss + (subcls_loss_weight*cls_200_loss)


        # _, predicted = torch.max(outputs, 1)
        # corrects += torch.sum(predicted == label_20)
        # total_samples += label_20.size(0)
        # loss = criterion(outputs, label_20)
        loss.backward()
        optimizer.step()
        global_counter += 1
        total_loss += 1.0*loss.item()
        total_samples += label_20.size(0)
        # torch.cuda.empty_cache()

        # print statistics
        if iter % disp_interval == disp_interval-1:    # print every X mini-batches
            print('k{} R{}| Ep:{} L:{} -20_LOSS:{} -200_LOSS:{} | -cls20:{} | -cls200_Top1:{} -Sum:{}'.format(k_cluster, round_nb, current_epoch, round(loss.item(), 3), round(cls_20_loss.item(), 3), round(cls_200_loss.item(), 3), avg_cls20_ep_acc, avg_ep_p_acc, avg_ep_acc))
            neptune.log_metric('train_loss', total_loss/total_samples)
            neptune.log_metric('lr', optimizer.param_groups[0]['lr'])
        avg_meter.add({'loss': cls_200_loss.item()})

    # average_accuracy = 100 * corrects / total_samples
    average_loss = total_loss / total_samples

    # val_loss = validate(model, criterion)
    # scheduler.step(val_loss)
    # neptune.log_metric('val_loss', val_loss)
    # print('Avg accuracy: ', average_accuracy)

    # Save every epoch
    save_checkpoint(
                    {'epoch': current_epoch,
                      'global_counter': global_counter,
                      'state_dict':model.state_dict(),
                      'optimizer':optimizer.state_dict()
                    },
                    filename='%s_epoch_%d.pth' %(dataset, current_epoch))
    
    # if current_epoch == epochs-1:
    #     save_checkpoint(
    #                     {
    #                         'epoch': current_epoch,
    #                         'global_counter': global_counter,
    #                         'state_dict':model.state_dict(),
    #                         'optimizer':optimizer.state_dict()
    #                     },
    #                     filename='%s_epoch_%d.pth' %(dataset, current_epoch))
    current_epoch += 1

print('Finished Training')

KeyboardInterrupt: ignored

In [ ]:
cls_20_loss

tensor(0.7192, device='cuda:0', grad_fn=<MeanBackward0>)

In [ ]:
mapp

{'dyed-lifted-polyps': 0,
 'dyed-resection-margins': 1,
 'esophagitis': 2,
 'normal-cecum': 3,
 'normal-pylorus': 4,
 'normal-z-line': 5,
 'polyps': 6,
 'ulcerative-colitis': 7}

In [ ]:
fname, l, label_20

(('/content/drive/My Drive/MAI/thesis/source/kvasir-dataset-v2-folds/1/test/normal-cecum/kvasir_v2__fold_1_test_class_normal-cecum_frame_2.jpg',
  '/content/drive/My Drive/MAI/thesis/source/kvasir-dataset-v2-folds/1/test/dyed-resection-margins/kvasir_v2__fold_1_test_class_dyed-resection-margins_frame_70.jpg',
  '/content/drive/My Drive/MAI/thesis/source/kvasir-dataset-v2-folds/1/test/ulcerative-colitis/kvasir_v2__fold_1_test_class_ulcerative-colitis_frame_50.jpg',
  '/content/drive/My Drive/MAI/thesis/source/kvasir-dataset-v2-folds/1/test/normal-pylorus/kvasir_v2__fold_1_test_class_normal-pylorus_frame_68.jpg'),
 tensor([3, 1, 7, 4]),
 tensor([[0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0.]], device='cuda:0'))

In [ ]:
F.one_hot(l[2], num_classes=8).numpy()

array([0, 0, 0, 0, 0, 0, 0, 1])

In [ ]:
filename = '/content/drive/MyDrive/MAI/thesis/source/kvasir-dataset-v2/normal-pylorus/d7011f89-92f1-4822-ae7e-5ba881c2da24.jpg'
img = PIL.Image.open(filename).convert("RGB")

In [ ]:
for img_name, img, label in train_loader:
  print(img_name)
  break

In [ ]:
img[ind,:,:].cpu().numpy().min()

In [ ]:
import matplotlib.pyplot as plt
ind = 15
t = img[ind,:,:].cpu().numpy()
t = t.transpose()

t2 = (t - t.min())/(t.max()-t.min())
plt.imshow(t2)
plt.title(CAT_LIST[torch.nonzero(label[ind])])

In [ ]:
CAT_LIST[torch.nonzero(label[0])]
